## Import and Load data file

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers  # Import layers directly from tensorflow.keras
from tensorflow.keras import ops  # Import ops directly from tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random


words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

c:\Users\harsh\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Pre-process data

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

lemmatize each word and remove duplicate words from the list

In [3]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

105 documents
22 classes ['cancel', 'confirm', 'delay', 'delivery', 'goodbye', 'greeting', 'items', 'mobile app', 'more', 'order status', 'payments', 'personal', 'profile info', 'reasons', 'refund', 'refund status', 'return', 'thanks', 'track', 'use voucher', 'user response', 'withdraw refunds']
145 unique lemmatized words ["'m", "'s", '.', 'a', 'about', 'accept', 'account', 'address', 'agent', 'am', 'an', 'any', 'anyone', 'app', 'are', 'available', 'be', 'become', 'but', 'buy', 'bye', 'call', 'can', 'cancel', 'cancelled', 'card', 'cash', 'change', 'common', 'completed', 'confirmation', 'confirmed', 'contact', 'credit', 'credited', 'customer', 'day', 'deducted', 'delay', 'delayed', 'delete', 'delivered', 'delivery', 'did', 'do', 'doe', 'doing', 'download', 'even', 'for', 'from', 'fund', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helpline', 'hey', 'hi', 'how', 'i', 'im', 'information', 'is', 'it', 'item', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'm

## Create training and testing data

In [5]:
import numpy as np

# Initialize training data and output
training = []
output_empty = [0] * len(classes)

# Iterate through documents
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # Create bag of words array with 1 if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # Output is '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    # Append bag of words and output to training data
    training.append([bag, output_row])

# Shuffle training data
random.shuffle(training)

# Convert training data to NumPy array
training = np.array(training, dtype="object")

# Split training data into X (patterns) and Y (intents)
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")


Training data created


## Create Model

In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD( decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0351 - loss: 3.1297       
Epoch 2/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 814us/step - accuracy: 0.0383 - loss: 3.0824   
Epoch 3/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 890us/step - accuracy: 0.1788 - loss: 2.9721
Epoch 4/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - accuracy: 0.1591 - loss: 2.9023
Epoch 5/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 816us/step - accuracy: 0.2419 - loss: 2.7233
Epoch 6/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.2090 - loss: 2.6838
Epoch 7/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.3415 - loss: 2.4882
Epoch 8/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - accuracy: 0.3214 - loss: 2.2916   
Epoch 9/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.4057 - loss: 2.1967
Epoch 10/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - accuracy: 0.4083 - loss: 1.9527
Epoch 11/200
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step - accuracy: 0.4921 - loss: 1.8201
Epoch 12/200
21/21 ━━━━━━━━━━━

model created


In [9]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [11]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [12]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [13]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# Main Code

In [14]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## GUI Interface

In [15]:
import tkinter
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#c5f0e3"
TEXT_COLOR = "#000000"

# BG_GRAY = "#ABB2B9"
# BG_COLOR = "#1c172a"
# TEXT_COLOR = "#ffffff"


FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"


def send(event):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#000000", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("E-Commerce Chatbot")
base.resizable(width=FALSE, height=FALSE)
base.configure(width=800, height=800, bg=BG_COLOR)


#Create Chat window
ChatLog = Text(base, bd=0, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT_BOLD)
ChatLog.config(state=DISABLED)

head_label = Label(base, bg=BG_COLOR, fg=TEXT_COLOR, text="Welcome to E-Commerce Chatbot", font=FONT_BOLD, pady=10)
head_label.place(relwidth=1)

line = Label(base, width=450, bg=BG_GRAY)


#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
ChatLog.focus()

#Create Button to send message
SendButton = Button(base, font=("Verdana", 12,'bold'), text="Send", width="12", height=15,
                    bd=0, bg="#ed9061", activebackground="#3c9d9b",fg='#ffffff',
                    command=lambda: send)

#Create the box to enter message
EntryBox = Text(base, bg="white",width="29", height="5", font="Arial", background="#dddddd")
EntryBox.focus()
EntryBox.bind("<Return>", send)
#EntryBox.bind("<Return>", send)

# bottom label
# bottom_label = Label(base, bg=BG_GRAY, height=80)
# bottom_label.place(relwidth=1, rely=0.825)
        
# message entry box
#EntryBox = Entry(bottom_label, fg=TEXT_COLOR, font=FONT)
# msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, |relx=0.011)
# msg_entry.focus()


#Place all components on the screen
# scrollbar.place(relheight=1, relx=0.974)
# ChatLog.place(relheight=1, width=1)
# EntryBox.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
# SendButton.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.20)\

scrollbar.place(x=775,y=6, height=800)
line.place(x=0,y=35, height=1, width=770)
ChatLog.place(x=5,y=40, height=700, width=770)
EntryBox.place(x=0, y=740, height=60, width=600)
SendButton.place(x=600, y=740, height=60, width=175)

In [16]:
base.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
